# Desafio LATAM

### Librerías

In [2]:
# instalar librerias
%pip install numpy
%pip install pandas

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


## Calidad de los datos

In [3]:
# importar librerias
import numpy as np
import pandas as pd

### Carga de datos

In [4]:
# cargar data y echar vistazo a datos
df = pd.read_csv('data/dataset_SCL.csv')
df.head()

<ipython-input-4-769d1d528a22>:2: DtypeWarning: Columns (1,6) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('data/dataset_SCL.csv')


,Fecha-I,Vlo-I,Ori-I,Des-I,Emp-I,Fecha-O,Vlo-O,Ori-O,Des-O,Emp-O,DIA,MES,AÑO,DIANOM,TIPOVUELO,OPERA,SIGLAORI,SIGLADES
0,2017-01-01 23:30:00,226,SCEL,KMIA,AAL,2017-01-01 23:33:00,226,SCEL,KMIA,AAL,1,1,2017,Domingo,I,American Airlines,Santiago,Miami
1,2017-01-02 23:30:00,226,SCEL,KMIA,AAL,2017-01-02 23:39:00,226,SCEL,KMIA,AAL,2,1,2017,Lunes,I,American Airlines,Santiago,Miami
2,2017-01-03 23:30:00,226,SCEL,KMIA,AAL,2017-01-03 23:39:00,226,SCEL,KMIA,AAL,3,1,2017,Martes,I,American Airlines,Santiago,Miami
3,2017-01-04 23:30:00,226,SCEL,KMIA,AAL,2017-01-04 23:33:00,226,SCEL,KMIA,AAL,4,1,2017,Miercoles,I,American Airlines,Santiago,Miami
4,2017-01-05 23:30:00,226,SCEL,KMIA,AAL,2017-01-05 23:28:00,226,SCEL,KMIA,AAL,5,1,2017,Jueves,I,American Airlines,Santiago,Miami


In [5]:
# chequear tipo de datos
df.dtypes

Fecha-I      object
Vlo-I        object
Ori-I        object
Des-I        object
Emp-I        object
Fecha-O      object
Vlo-O        object
Ori-O        object
Des-O        object
Emp-O        object
DIA           int64
MES           int64
AÑO           int64
DIANOM       object
TIPOVUELO    object
OPERA        object
SIGLAORI     object
SIGLADES     object
dtype: object

In [6]:
# existen filas duplicadas?
df.duplicated().any()

False

In [7]:
# shape
df.shape

(68206, 18)

In [8]:
# columnas con valores no definidos
df.isna().any()

Fecha-I      False
Vlo-I        False
Ori-I        False
Des-I        False
Emp-I        False
Fecha-O      False
Vlo-O         True
Ori-O        False
Des-O        False
Emp-O        False
DIA          False
MES          False
AÑO          False
DIANOM       False
TIPOVUELO    False
OPERA        False
SIGLAORI     False
SIGLADES     False
dtype: bool

* Existen valores no definidos en columna `Vlo-O`. ¿Cuantos serán?

In [9]:
df['Vlo-O'].isna().sum()

1

### Liempieza de datos

* Cambiar tipo de datos de columna: \
    `- Fecha-I      timestamp` \
    `- Vlo-I        int` \
    `- Ori-I        object` \
    `- Des-I        object` \
    `- Emp-I        object` \
    `- Fecha-O      timestamp` \
    `- Vlo-O         int` \
    `- Ori-O        object` \
    `- Des-O        object` \
    `- Emp-O        object` \
    `- DIA          int` \
    `- MES          int` \
    `- AÑO          int` \
    `- DIANOM       object` \
    `- TIPOVUELO    object` \
    `- OPERA        object` \
    `- SIGLAORI     object` \
    `- SIGLADES     object` 
    
* Quitar fila sin dato en columna `Vlo-O`. Tambien quitar filas relacionadas con ésta, de forma que no tenga impacto en analisis

In [10]:
# crear copia
df_clean = df.copy()

In [11]:
# df_clean[df_clean[['Fecha-I', 'Vlo-I']].duplicated(keep=False)].sort_values(by=['Fecha-I', 'Vlo-I'])

In [12]:
# cambio de tipo de datos
df_clean = df.copy()
df_clean['Fecha-I'] = pd.to_datetime(df_clean['Fecha-I'])
df_clean['Fecha-O'] = pd.to_datetime(df_clean['Fecha-O'])
# df_clean['Vlo-I'].astype('int')

In [13]:
df_clean.dtypes

Fecha-I      datetime64[ns]
Vlo-I                object
Ori-I                object
Des-I                object
Emp-I                object
Fecha-O      datetime64[ns]
Vlo-O                object
Ori-O                object
Des-O                object
Emp-O                object
DIA                   int64
MES                   int64
AÑO                   int64
DIANOM               object
TIPOVUELO            object
OPERA                object
SIGLAORI             object
SIGLADES             object
dtype: object

* Cambiar `Vlo-I` y `Vlo-O` a `int`

In [14]:
df_clean['Vlo-I'] = df_clean['Vlo-I'].astype(int)

ValueError: invalid literal for int() with base 10: '989P'

* Identificar strings que no se pueden convertir a int

In [15]:
df_clean['Vlo-I'].to_list()

num_buenos = []
num_malos = []
for i, vlo_i_i in enumerate(df_clean['Vlo-I'].to_list()):
    try:
        num_buenos.append(int(vlo_i_i))
    except:
        num_malos.append(vlo_i_i)          
print(len(num_buenos))

68201


In [16]:
print(num_malos)

['989P', '940P', '591P', '846A', '1104A']


* Existen numeros con character. ¿Será un error de tipeo? Para validar esto se quitará el ultimo char en las filas de estos casos y se comparara versus el resto del dataset para ver si se duplican filas. En caso de no duplicarse filas significa que fue un error de tipeo

In [17]:
# seleccionar dataset con char
df_clean_w_char = df_clean[df_clean['Vlo-I'].isin(num_malos)].copy()

# quitarles ultimo caracter 
df_clean_w_char['Vlo-I'] = df_clean_w_char['Vlo-I'].str[:-1] # quitarles ultimo caracter 
df_clean_w_char

,Fecha-I,Vlo-I,Ori-I,Des-I,Emp-I,Fecha-O,Vlo-O,Ori-O,Des-O,Emp-O,DIA,MES,AÑO,DIANOM,TIPOVUELO,OPERA,SIGLAORI,SIGLADES
22232,2017-05-13 21:50:00,989,SCEL,SUMU,AAL,2017-05-13 21:52:00,989,SCEL,SUMU,AAL,13,5,2017,Sabado,I,American Airlines,Santiago,Montevideo
27464,2017-06-16 20:30:00,940,SCEL,KDFW,AAL,2017-06-16 20:50:00,940,SCEL,KDFW,AAL,16,6,2017,Viernes,I,American Airlines,Santiago,Dallas
39225,2017-08-07 19:00:00,591,SCEL,LFPG,PUE,2017-08-07 20:35:00,591P,SCEL,LFPG,PUE,7,8,2017,Lunes,I,Plus Ultra Lineas Aereas,Santiago,Paris
39266,2017-08-24 23:00:00,846,SCEL,KIAH,UAL,2017-08-24 23:00:00,2804,SCEL,KIAH,UAL,24,8,2017,Jueves,I,United Airlines,Santiago,Houston
58126,2017-11-05 17:00:00,1104,SCEL,SCSE,SKU,2017-11-05 17:42:00,1104,SCEL,SCSE,SKU,5,11,2017,Domingo,N,Sky Airline,Santiago,La Serena


In [18]:
# seleccionar el complemento
# quitarles ultimo caracter 
df_complement = df_clean[~df_clean['Vlo-I'].isin(num_malos)].copy()

In [19]:
# cuadratura
df_complement.shape, df_clean_w_char.shape, df.shape

((68201, 18), (5, 18), (68206, 18))

In [20]:
df_append = pd.concat([df_complement, df_clean_w_char])

In [21]:
df_append.head()

,Fecha-I,Vlo-I,Ori-I,Des-I,Emp-I,Fecha-O,Vlo-O,Ori-O,Des-O,Emp-O,DIA,MES,AÑO,DIANOM,TIPOVUELO,OPERA,SIGLAORI,SIGLADES
0,2017-01-01 23:30:00,226,SCEL,KMIA,AAL,2017-01-01 23:33:00,226,SCEL,KMIA,AAL,1,1,2017,Domingo,I,American Airlines,Santiago,Miami
1,2017-01-02 23:30:00,226,SCEL,KMIA,AAL,2017-01-02 23:39:00,226,SCEL,KMIA,AAL,2,1,2017,Lunes,I,American Airlines,Santiago,Miami
2,2017-01-03 23:30:00,226,SCEL,KMIA,AAL,2017-01-03 23:39:00,226,SCEL,KMIA,AAL,3,1,2017,Martes,I,American Airlines,Santiago,Miami
3,2017-01-04 23:30:00,226,SCEL,KMIA,AAL,2017-01-04 23:33:00,226,SCEL,KMIA,AAL,4,1,2017,Miercoles,I,American Airlines,Santiago,Miami
4,2017-01-05 23:30:00,226,SCEL,KMIA,AAL,2017-01-05 23:28:00,226,SCEL,KMIA,AAL,5,1,2017,Jueves,I,American Airlines,Santiago,Miami


In [22]:
df_append.duplicated().any()

False

* Al quitar ultimo char y volver a concatenar resultados no hay filas duplicadas. Los datos quedan limpios. Se puede volver a limpiar las columnas

In [23]:
df_append.dtypes

Fecha-I      datetime64[ns]
Vlo-I                object
Ori-I                object
Des-I                object
Emp-I                object
Fecha-O      datetime64[ns]
Vlo-O                object
Ori-O                object
Des-O                object
Emp-O                object
DIA                   int64
MES                   int64
AÑO                   int64
DIANOM               object
TIPOVUELO            object
OPERA                object
SIGLAORI             object
SIGLADES             object
dtype: object

In [24]:
df_append['Vlo-I'] = df_append['Vlo-I'].astype('int')

* Repetir proceso para columna `Vlo-O`

In [25]:
df_append['Vlo-O'].to_list()

num_buenos = []
num_malos = []
for i, vlo_i_i in enumerate(df_append['Vlo-O'].to_list()):
    try:
        num_buenos.append(float(vlo_i_i))
    except:
        num_malos.append(vlo_i_i)          
# print(len(num_buenos))

In [26]:
num_malos

['71R',
 '71R',
 '201R',
 '71R',
 '43R',
 '401A',
 '401B',
 '802R',
 '114R',
 '622R',
 '492R',
 '338R',
 '846A',
 '502R',
 '207R',
 '591P']

In [27]:
# seleccionar dataset con char
df_clean_w_char = df_append[df_append['Vlo-O'].isin(num_malos)].copy()

# quitarles ultimo caracter 
df_clean_w_char['Vlo-O'] = df_clean_w_char['Vlo-O'].str[:-1] # quitarles ultimo caracter 
df_clean_w_char

,Fecha-I,Vlo-I,Ori-I,Des-I,Emp-I,Fecha-O,Vlo-O,Ori-O,Des-O,Emp-O,DIA,MES,AÑO,DIANOM,TIPOVUELO,OPERA,SIGLAORI,SIGLADES
13906,2017-03-30 10:30:00,71,SCEL,SCIE,SKU,2017-03-30 10:50:00,71,SCEL,SCIE,SKU,30,3,2017,Jueves,N,Sky Airline,Santiago,Concepcion
13907,2017-03-22 11:00:00,1071,SCEL,SCIE,SKU,2017-03-22 11:00:00,71,SCEL,SCIE,SKU,22,3,2017,Miercoles,N,Sky Airline,Santiago,Concepcion
17055,2017-03-22 10:00:00,201,SCEL,SCIE,LXP,2017-03-22 11:50:00,201,SCEL,SCIE,LXP,22,3,2017,Miercoles,N,Grupo LATAM,Santiago,Concepcion
19207,2017-04-25 09:00:00,71,SCEL,SCIE,SKU,2017-04-25 09:29:00,71,SCEL,SCIE,SKU,25,4,2017,Martes,N,Sky Airline,Santiago,Concepcion
22167,2017-04-06 21:10:00,43,SCEL,SCIE,LAW,2017-04-06 21:24:00,43,SCEL,SCIE,JMR,6,4,2017,Jueves,N,Latin American Wings,Santiago,Concepcion
22301,2017-05-26 09:40:00,401,SCEL,LFPG,AFR,2017-05-26 10:03:00,401,SCEL,LFPG,AFR,26,5,2017,Viernes,I,Air France,Santiago,Paris
22302,2017-05-28 09:40:00,401,SCEL,LFPG,AFR,2017-05-28 09:43:00,401,SCEL,LFPG,AFR,28,5,2017,Domingo,I,Air France,Santiago,Paris
24304,2017-05-11 10:00:00,802,SCEL,SPJC,SKU,2017-05-11 10:36:00,802,SCEL,SPJC,SKU,11,5,2017,Jueves,I,Sky Airline,Santiago,Lima
27246,2017-05-15 12:15:00,114,SCEL,SCAT,LAN,2017-05-15 13:10:00,114,SCEL,SCAT,LAN,15,5,2017,Lunes,N,Grupo LATAM,Santiago,Copiapo
27247,2017-05-25 13:15:00,622,SCEL,MMMX,LAN,2017-05-25 13:23:00,622,SCEL,MMMX,LAN,25,5,2017,Jueves,I,Grupo LATAM,Santiago,Ciudad de Mexico


In [28]:
# seleccionar el complemento
# quitarles ultimo caracter 
df_complement = df_append[~df_append['Vlo-O'].isin(num_malos)].copy()

In [29]:
# cuadratura
df_complement.shape, df_clean_w_char.shape, df.shape

((68190, 18), (16, 18), (68206, 18))

In [30]:
df_append_2 = pd.concat([df_complement, df_clean_w_char])

In [31]:
df_append_2.duplicated().any()

False

In [32]:
df_append_2['Vlo-O'] = df_append_2['Vlo-O'].astype('int')

ValueError: cannot convert float NaN to integer

In [33]:
# quitar dato con NaN
df_append_2.dropna(inplace=True)

In [34]:
# pasar datos a float primero
df_append_2['Vlo-O'] = df_append_2['Vlo-O'].astype('float')

In [35]:
# luego a int
df_append_2['Vlo-O'] = df_append_2['Vlo-O'].astype('int')

In [36]:
# chequear tipo de datos
df_append_2.dtypes

Fecha-I      datetime64[ns]
Vlo-I                 int64
Ori-I                object
Des-I                object
Emp-I                object
Fecha-O      datetime64[ns]
Vlo-O                 int64
Ori-O                object
Des-O                object
Emp-O                object
DIA                   int64
MES                   int64
AÑO                   int64
DIANOM               object
TIPOVUELO            object
OPERA                object
SIGLAORI             object
SIGLADES             object
dtype: object

In [37]:
# comparacion datasets
df_append_2.duplicated().any(), df_append_2.shape

(False, (68205, 18))

In [38]:
df.duplicated().any(), df.shape

(False, (68206, 18))

* Terminamos con un dataframe con los tipos de datos propuestos y con una fila menos, que es aquella que tiene valor nulo en la columna `Vlo-O`.
* Fin de la limpieza de datos. Comienzo de analisis

## Analisis de datos 

In [39]:
dataset = df_append_2.copy()

In [41]:
dataset.dtypes

Fecha-I      datetime64[ns]
Vlo-I                 int64
Ori-I                object
Des-I                object
Emp-I                object
Fecha-O      datetime64[ns]
Vlo-O                 int64
Ori-O                object
Des-O                object
Emp-O                object
DIA                   int64
MES                   int64
AÑO                   int64
DIANOM               object
TIPOVUELO            object
OPERA                object
SIGLAORI             object
SIGLADES             object
dtype: object